In [ ]:
import geopandas
import rasterio
from pathlib import Path
import numpy as np
import pandas as pd
from rasterstats import zonal_stats
import matplotlib.pyplot as plt
from ipywidgets import interact

from utils import read_raster, reproject_raster, compute_area, zonal_stats_for_value

%load_ext autoreload
%autoreload 2

In [ ]:
DATA_PATH = Path("data/")
CRS = "EPSG:4326"

### Concentration des fermes et des entailles

In [ ]:
df_farms = pd.concat(
    [
        pd.read_csv(DATA_PATH / "fermes" / "3210036201_donneesselectionnees.csv", delimiter=";"), 
        pd.read_csv(DATA_PATH / "fermes" / "3210042301_donneesselectionnees.csv", delimiter=";")
    ]
).drop(
    [
        "UNITÉ DE MESURE", 
        "IDENTIFICATEUR D'UNITÉ DE MESURE", 
        "FACTEUR SCALAIRE", 
        "VECTEUR", 
        "COORDONNÉES", 
        "IDENTIFICATEUR SCALAIRE", 
        "STATUS", 
        "SYMBOLE", 
        "TERMINÉ", 
        "DÉCIMALES"
    ], 
    axis=1
).reset_index()

df_farms["GÉO"] = df_farms.apply(
    lambda row: row["GÉO"].split(" [")[0], 
    axis=1
)

df_farms["DGUID"] = df_farms.apply(
    lambda row: row["DGUID"][4:], 
    axis=1
)


df_farms = df_farms.replace(
    "Nombre d'exploitations déclarantes", 
    "num_exploitations"
).replace(
    "Nombre d'entailles", 
    "num_entailles"
).replace(
    "Nombre de fermes déclarantes", 
    "num_exploitations"
).pivot(
    index=["GÉO", "DGUID"], 
    columns=["PÉRIODE DE RÉFÉRENCE", "Entailles d'érables"], 
    values="VALEUR"
)

df_farms.columns = [f"{x}_{y}" for x, y in df_farms.columns.to_flat_index()]

df_farms.head()

### Fichiers des limites géographiques

In [ ]:
df_boundaries = geopandas.read_file(
    DATA_PATH / "boundaries" / "lcar000b21a_e.shp"
).to_crs(
    CRS
).drop(
    ["CARUID", "CARENAME", "LANDAREA"], 
    axis=1
).rename(
    columns={"CARFNAME": "area_name"}
)

df_boundaries["DGUID"] = df_boundaries.apply(
    lambda row: row["DGUID"][4:], 
    axis=1
)

df_boundaries["total_area"] = compute_area(df_boundaries)

df_boundaries.head()

In [ ]:
df_boundaries.plot("total_area")

### Carte de la biomasse courante

In [ ]:
filename = DATA_PATH / "cartography" / "ACESAC_volume_250m_final-100.tif"    
biomass_raster, affine, no_data_value = read_raster(filename, crs=CRS)
    
plt.imshow(biomass_raster)
plt.show()

In [ ]:
df_boundaries["stats"] = zonal_stats(
    vectors=df_boundaries['geometry'], 
    raster=biomass_raster, 
    affine=affine, 
    stats="sum", 
    nodata=no_data_value
)

df_boundaries["total_biomass"] = df_boundaries.apply(
    lambda row: row["stats"]["sum"], 
    axis=1
)

df_boundaries = df_boundaries.drop(["stats"], axis=1)

df_boundaries.head()

### Déplacement des zones propices 

In [ ]:
filename = DATA_PATH / "rasters" / "arcp8510000532011-2040.asc"

raster_2011_2040, affine, no_data_value = read_raster(filename, crs=CRS)
    
plt.imshow(raster_2011_2040)
plt.show()

In [ ]:
period = "2011-2040"

for value, name in [
    (1, "marginal"), 
    (3, "core"), 
    (no_data_value, "all")
]:
    df_boundaries[f"{name}_{period}"] = zonal_stats_for_value(
        raster=raster_2011_2040, 
        vectors=df_boundaries["geometry"], 
        value=value, 
        stats="count", 
        data_value=100, 
        no_data_value=no_data_value, 
        affine=affine
    )

In [ ]:
filename = DATA_PATH / "rasters" / "arcp8510000532041-2070.asc"

raster_2041_2070, affine, no_data_value = read_raster(filename, crs=CRS)
    
plt.imshow(raster_2041_2070)
plt.show()

In [ ]:
period = "2041-2070"

for value, name in [
    (1, "marginal"), 
    (3, "core"), 
    (no_data_value, "all")
]:
    df_boundaries[f"{name}_{period}"] = zonal_stats_for_value(
        raster=raster_2041_2070, 
        vectors=df_boundaries["geometry"], 
        value=value, 
        stats="count", 
        data_value=100, 
        no_data_value=no_data_value, 
        affine=affine
    )

In [ ]:
filename = DATA_PATH / "rasters" / "arcp8510000532071-2100.asc"
raster_2071_2100, affine, no_data_value = read_raster(filename, crs=CRS)
    
plt.imshow(raster_2071_2100)
plt.show()

In [ ]:
period = "2071-2100"

for value, name in [
    (1, "marginal"), 
    (3, "core"), 
    (no_data_value, "all")
]:
    df_boundaries[f"{name}_{period}"] = zonal_stats_for_value(
        raster=raster_2071_2100, 
        vectors=df_boundaries["geometry"], 
        value=value, 
        stats="count", 
        data_value=100, 
        no_data_value=no_data_value, 
        affine=affine
    )

### On combine les deux dataframes en un seul 

In [ ]:
df = pd.merge(
    df_boundaries, 
    df_farms, 
    on="DGUID"
)

df.head()

In [ ]:
def _plot(pr_uid, column_name):
    return df[df_boundaries["PRUID"] == pr_uid].explore(column_name)
    
interact(
    _plot,
    pr_uid= df["PRUID"].unique(),
    column_name=df.columns.tolist()[4:]
)